In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from pandas import ExcelWriter
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)
%matplotlib inline

In [40]:
location='выборка.xlsx'
df=pd.read_excel(location)

In [41]:
df.columns

Index(['№ страхового события', 'Участник 1', 'Участник 2'], dtype='object')

In [42]:
df=df.drop('№ страхового события', axis=1)

Смотрим на наличие пропусков

In [43]:
df.isnull().values.any()

False

Смотрим, кто был во многих ДТП

In [44]:
df['Участник 2'].value_counts().head()

Дорофеева Гулия Николаевна       2
Мамонтов Мстислав Георгиевич     2
Сенчукова Екатерина Семеновна    2
Воробьев Иван Александрович      1
Мамонтова Саида Никифоровна      1
Name: Участник 2, dtype: int64

In [45]:
df['Участник 1'].value_counts().head()

Дорофеева Гулия Николаевна         3
Стрелков Евгений Егорович          2
Котова Виктория Архиповна          2
Мухамадеев Александр Валерьевич    2
Дмитриев Радик Иванович            1
Name: Участник 1, dtype: int64

Дорофеева Гулия Николаевна была несколько раз и 1 и 2 участником, скорее всего фрод

Разобьем ФИО на составляющие и проверим, были ли ДТП с одинаковыми фамилиями, возможно это родственники, которые подстроили ДТП

In [46]:
df1=df['Участник 1'].str.split(' ', expand=True)
df1.columns=['S1', 'N1', 'P1']
df2=df['Участник 2'].str.split(' ', expand=True)
df2.columns=['S2', 'N2', 'P2']

In [47]:
df = pd.concat([df1, df2], axis=1)

In [48]:
df.head()

,S1,N1,P1,S2,N2,P2
0,Беляев,Владислав,Аркадьевич,Ефимов,Малик,Константинович
1,Давыдова,Ануш,Оскаровна,Никифорова,Божена,Львовна
2,Белов,Сергей,Михайлович,Третьяков,Никита,Харитонович
3,Власов,Артём,Михайлович,Аксенова,Вероника,Кузьминична
4,Яковлев,Алан,Макарович,Сысоев,Елисей,Тимофеевич


In [50]:
df[df.S1==df.S2]

,S1,N1,P1,S2,N2,P2


In [53]:
df[df.N1==df.N2]

,S1,N1,P1,S2,N2,P2


In [52]:
df[df.P1==df.P2]

,S1,N1,P1,S2,N2,P2
252,Мишин,Ратибор,Денисович,Алексеев,Игорь,Денисович


Нет совпадающих фамилий и даже имён, есть одно отчестно, но это похоже на совпадение, надо проверить схожие фамилии, удалим букву "а" с конца фамилий

In [76]:
for i in range(len(df.S1)):
    if df.S1[i][-1]=='а':
        df.S1[i]=df.S1[i][:-1]
for i in range(len(df.S2)):
    if df.S2[i][-1]=='а':
        df.S2[i]=df.S2[i][:-1]

In [78]:
df[df.S1==df.S2]

,S1,N1,P1,S2,N2,P2


Смотрим популярные Фамилии без пола, снова Дорофеев, еще добавились Стрелков, Павлов, Мамонтов, но возможно, что это совпадение

In [83]:
print (df.S1.value_counts()[:10], '\n')
print (df.S2.value_counts()[:10])

Павлов          3
Стрелков        3
Дорофеев        3
Ситников        2
Шестаков        2
Морозов         2
Красильников    2
Селиверстов     2
Кириллов        2
Савельев        2
Name: S1, dtype: int64 

Дорофеев      3
Мамонтов      3
Семенов       2
Беляков       2
Калашников    2
Чернов        2
Степанов      2
Пахомов       2
Белоусов      2
Лихачев       2
Name: S2, dtype: int64


In [85]:
df[df.S1==df.S2]

,S1,N1,P1,S2,N2,P2


Всё равно не нашли тех, кто мог подстроить ДТП с родственниками